In [1]:
# 전자결재 문서ID 추출기
# 특정 기간의 문서ID만 텍스트 파일로 추출

"""
 [기능]
 전자결재 시스템에서 특정 기간의 문서ID만 추출하여 txt 파일로 저장

 [출력 형식]
 "27444","28456","29777" (한 줄에 쌍따옴표+쉼표 구분)

 [동작 순서]
 1. 브라우저 실행
 2. 수동 작업 대기 (로그인 → 메뉴 진입 → 필터 설정 → 조회)
 3. 그리드에서 문서ID 추출
 4. 다음 페이지로 이동하며 반복
 5. txt 파일로 저장

 [설정]
 - START_DATE / END_DATE: 조회 기간
 - OUTPUT_FILE: 출력 파일명
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# ═══════════════════════════════════════════════════════════
# 🔧 설정 (여기만 수정하세요!)
# ═══════════════════════════════════════════════════════════

BASE_URL = "http://office.anyfive.com/home/"

# 기간 설정
START_DATE = "2010-01-01"  # 여기를 수정하세요 - 시작일
END_DATE = "2013-12-31"    # 여기를 수정하세요 - 종료일

# START_DATE = "2014-01-01"  # 여기를 수정하세요 - 시작일
# END_DATE = "2017-12-31"    # 여기를 수정하세요 - 종료일

# START_DATE = "2018-01-01"  # 여기를 수정하세요 - 시작일
# END_DATE = "2021-12-31"    # 여기를 수정하세요 - 종료일

# START_DATE = "2022-01-01"  # 여기를 수정하세요 - 시작일
# END_DATE = "2025-12-31"    # 여기를 수정하세요 - 종료일

# 출력 파일명
OUTPUT_FILE = "doc_ids_2013.txt"  # 여기를 수정하세요

# ═══════════════════════════════════════════════════════════
# 📦 유틸리티 함수
# ═══════════════════════════════════════════════════════════

def switch_into_frame_having(driver, selector_css, max_retries=3):
    """selector가 있는 프레임으로 전환"""
    for attempt in range(max_retries):
        driver.switch_to.default_content()
        
        # 로딩 마스크 대기
        try:
            WebDriverWait(driver, 5).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
            )
        except:
            pass
        
        time.sleep(1)
        
        # 최상위 확인
        if driver.find_elements(By.CSS_SELECTOR, selector_css):
            print(f"  ✓ 그리드 발견 (최상위)")
            return True
        
        # 프레임 내부 탐색
        frames = driver.find_elements(By.CSS_SELECTOR, "iframe, frame")
        for idx, fr in enumerate(frames):
            try:
                driver.switch_to.default_content()
                driver.switch_to.frame(fr)
                if driver.find_elements(By.CSS_SELECTOR, selector_css):
                    print(f"  ✓ 그리드 발견 (프레임 {idx})")
                    return True
            except:
                continue
        
        if attempt < max_retries - 1:
            print(f"  ⚠️ 재시도 {attempt + 1}/{max_retries}")
            time.sleep(2)
    
    return False

def wait_grid_stable(driver, timeout=10):
    """그리드 안정화 대기"""
    try:
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
        )
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table[role='grid'] tbody tr"))
        )
        time.sleep(1)
        return True
    except:
        return False

# ═══════════════════════════════════════════════════════════
# 🕷️ 문서ID 추출
# ═══════════════════════════════════════════════════════════

def extract_doc_ids(driver):
    """현재 페이지의 문서ID 추출"""
    table = driver.find_element(By.CSS_SELECTOR, "table[role='grid']")
    rows = table.find_elements(By.CSS_SELECTOR, "tbody tr[role='row']")
    
    doc_ids = []
    for tr in rows:
        cb = tr.find_elements(By.CSS_SELECTOR, "td input[name='aprDoc_grid_ckbx']")
        if cb:
            doc_id = cb[0].get_attribute("value").strip()
            if doc_id:
                doc_ids.append(doc_id)
    
    return doc_ids

# ═══════════════════════════════════════════════════════════
# 🔄 페이징
# ═══════════════════════════════════════════════════════════

def has_next_page(driver):
    """다음 페이지 존재 여부"""
    try:
        next_btns = driver.find_elements(By.CSS_SELECTOR, 
            "a.k-pager-nav[aria-label='다음 페이지로 이동']:not(.k-state-disabled)")
        return len(next_btns) > 0
    except:
        return False

def go_to_next_page(driver):
    """다음 페이지로 이동"""
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR,
            "a.k-pager-nav[aria-label='다음 페이지로 이동']:not(.k-state-disabled)")
        
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)
        
        WebDriverWait(driver, 15).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
        )
        
        wait_grid_stable(driver)
        return True
    except Exception as e:
        print(f"  ❌ 페이지 이동 실패: {e}")
        return False

# ═══════════════════════════════════════════════════════════
# 🚀 메인
# ═══════════════════════════════════════════════════════════

def main():
    print("=" * 60)
    print("🔍 전자결재 문서ID 추출기")
    print(f"📅 기간: {START_DATE} ~ {END_DATE}")
    print(f"📄 출력: {OUTPUT_FILE}")
    print("=" * 60)
    
    # 브라우저 시작
    opts = webdriver.ChromeOptions()
    opts.add_argument("--start-maximized")
    opts.add_argument("--disable-popup-blocking")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    
    driver = webdriver.Chrome(service=Service(), options=opts)
    
    try:
        driver.get(BASE_URL)
        time.sleep(1)
        
        print("\n" + "=" * 60)
        print("📋 수동 작업이 필요합니다!")
        print("=" * 60)
        print("다음 단계를 완료한 후 Enter를 눌러주세요:\n")
        print("  1️⃣  로그인")
        print("  2️⃣  전자결재 메뉴 클릭")
        print("  3️⃣  결재 문서관리 클릭")
        print(f"  4️⃣  상태 = 완료 선택")
        print(f"  5️⃣  시작일 = {START_DATE}")
        print(f"  6️⃣  종료일 = {END_DATE}")
        print("  7️⃣  조회 버튼 클릭 → 그리드 나타남 확인")
        print("\n💡 그리드가 화면에 보이면 Enter를 누르세요!")
        print("=" * 60)
        input("\n✋ 준비되면 Enter... ")
        
        time.sleep(1)
        
        # 그리드로 전환
        if not switch_into_frame_having(driver, "table[role='grid']"):
            print("❌ 그리드를 찾을 수 없습니다")
            return
        
        wait_grid_stable(driver)
        
        # 전체 문서ID 수집
        all_doc_ids = []
        current_page = 1
        
        while True:
            print(f"\n📄 페이지 {current_page} 처리 중...")
            
            # 그리드 재확인
            if not switch_into_frame_having(driver, "table[role='grid']"):
                print("⚠️ 그리드 재탐색 실패")
                break
            
            wait_grid_stable(driver)
            
            # 문서ID 추출
            page_ids = extract_doc_ids(driver)
            
            if not page_ids:
                print("⚠️ 데이터가 없습니다")
                break
            
            all_doc_ids.extend(page_ids)
            print(f"  ✓ {len(page_ids)}건 추출 (누적: {len(all_doc_ids)}건)")
            
            # 다음 페이지
            if has_next_page(driver):
                if go_to_next_page(driver):
                    current_page += 1
                else:
                    break
            else:
                print(f"\n🎉 마지막 페이지 도달!")
                break
        
        # txt 파일 저장
        if all_doc_ids:
            output_text = ",".join(f'"{doc_id}"' for doc_id in all_doc_ids)
            with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
                f.write(output_text)
            
            print(f"\n{'=' * 60}")
            print(f"✅ 완료!")
            print(f"{'=' * 60}")
            print(f"📊 총 {len(all_doc_ids)}건 추출")
            print(f"📄 저장: {OUTPUT_FILE}")
            print(f"{'=' * 60}")
        else:
            print("\n⚠️ 추출된 문서ID가 없습니다")
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
    
    finally:
        input("\n✋ 브라우저를 닫으려면 Enter를 누르세요...")
        driver.quit()

if __name__ == "__main__":
    main()

🔍 전자결재 문서ID 추출기
📅 기간: 2010-01-01 ~ 2013-12-31
📄 출력: doc_ids_2013.txt

📋 수동 작업이 필요합니다!
다음 단계를 완료한 후 Enter를 눌러주세요:

  1️⃣  로그인
  2️⃣  전자결재 메뉴 클릭
  3️⃣  결재 문서관리 클릭
  4️⃣  상태 = 완료 선택
  5️⃣  시작일 = 2010-01-01
  6️⃣  종료일 = 2013-12-31
  7️⃣  조회 버튼 클릭 → 그리드 나타남 확인

💡 그리드가 화면에 보이면 Enter를 누르세요!



✋ 준비되면 Enter...  


  ✓ 그리드 발견 (프레임 1)

📄 페이지 1 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 50건)

📄 페이지 2 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 100건)

📄 페이지 3 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 150건)

📄 페이지 4 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 200건)

📄 페이지 5 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 250건)

📄 페이지 6 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 300건)

📄 페이지 7 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 350건)

📄 페이지 8 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 400건)

📄 페이지 9 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 450건)

📄 페이지 10 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 500건)

📄 페이지 11 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 550건)

📄 페이지 12 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 600건)

📄 페이지 13 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 650건)

📄 페이지 14 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 700건)

📄 페이지 15 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 750건)

📄 페이지 16 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출 (누적: 800건)

📄 페이지 17 처리 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 추출


✋ 브라우저를 닫으려면 Enter를 누르세요... 
